In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import  StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense,LeakyReLU,LSTM,Input,Lambda,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import Accuracy
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from concurrent.futures import ThreadPoolExecutor


In [2]:
data=pd.read_csv('../datasets/PSD45.csv').iloc[:,1:33]
labels=pd.read_csv('../datasets/Labels.csv').iloc[:,1:4]

In [3]:
data

,Fp1,AF3,F3,F7,FC5,FC1,C3,T7,CP5,CP1,...,FC2,Cz,C4,T8,CP6,CP2,P4,P8,PO4,O2
0,0.331383,0.302093,0.134594,1.026736,0.467752,0.204260,0.175026,0.226679,0.312170,0.315429,...,0.296733,0.143684,0.390341,0.546754,0.816526,0.353447,0.416153,0.209551,0.112324,0.584553
1,0.256327,0.093801,0.209467,0.520343,0.332835,0.160808,0.244056,0.422417,0.117003,0.411794,...,0.322102,0.828927,0.391170,0.090605,0.150150,0.887933,0.467302,0.167608,0.480559,0.412893
2,0.311773,0.392808,0.322142,0.378091,0.162233,0.311832,0.632373,0.708174,0.366250,0.416871,...,0.331855,1.011100,0.453283,0.292421,0.394736,0.388968,0.122635,1.107560,0.664384,0.253671
3,0.165655,0.255653,0.437500,0.348375,0.721522,0.412234,0.100055,0.176210,1.006885,0.485257,...,0.436905,0.289155,0.158386,0.895881,0.212445,0.117028,0.350288,0.498311,0.421862,0.171673
4,0.256001,0.636777,0.134918,0.462233,1.061191,0.529741,0.275045,0.412421,0.402623,0.112826,...,0.347010,0.110870,0.572758,0.477297,0.302808,0.353191,0.420857,0.401010,0.136428,0.438557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57595,0.208354,5.899343,0.786404,0.156360,2.303209,0.865383,13.545904,0.952523,1.426254,11.053223,...,41.810952,7.126157,0.473278,0.446589,4.063789,1.143567,0.385316,7.531276,2.832202,0.637985
57596,0.428503,5.576647,9.890570,1.496358,6.629244,2.367740,0.573862,2.181876,2.941306,1.286554,...,6.574940,1.157410,236.302717,3.790158,0.969567,0.677345,4.711672,41.231050,3.268565,16.743982
57597,2.349644,1.666489,0.866412,40.515346,4.597316,0.415907,0.753588,4.249526,1.418109,0.421669,...,1.117740,1.573215,1.764416,0.591828,0.197271,5.995950,0.558259,0.308661,2.478292,1.161655
57598,8.478914,0.946635,1.253631,4.861493,1.304811,134.234555,3.838969,1.141114,0.535277,3.653305,...,0.386376,20.073992,3.009233,0.736745,0.392250,4.558863,14.405906,3.604771,7.266194,2.578681


In [4]:
labels

,Valence,Arousal,Dominance
0,7.71,7.60,6.90
1,8.10,7.31,7.28
2,8.58,7.54,9.00
3,4.94,6.01,6.12
4,6.96,3.92,7.19
...,...,...,...
1275,3.91,6.96,5.82
1276,2.81,6.13,6.06
1277,3.05,7.01,5.10
1278,3.99,7.17,4.85


In [5]:
#cols=['Fp1','F3','F7','C3','T7','CP5','P3','O1','Fp2','F8','T8'] #this is for feature selection
cols=data.columns
scaler=StandardScaler()
scaled_data=pd.DataFrame(scaler.fit_transform(data[cols]),columns=cols)

In [6]:
scaled_labels=labels>=5
encoder=LabelEncoder()
encoded_labels=[]
encoded_labels.append(encoder.fit_transform(scaled_labels.iloc[:,0]))# 0 for valence and 1 for arousal
encoded_labels=pd.DataFrame(np.array(encoded_labels).transpose(),columns=['label'])

In [7]:
scaled_data=scaled_data.to_numpy().reshape(1280,45,32)
encoded_labels = to_categorical(encoded_labels, num_classes=2)

In [8]:
class SpinalLSTM:

    def __init__(self,shape:tuple,units:list,steps:int,features_per_lyr=1):
        super().__init__()
        self.shape=shape
        self.hl_units=units
        self.time_steps=steps
        self.features_per_lyr=features_per_lyr
        self.reps=int(len(self.hl_units)/self.shape[0])
        self.lyrs=int(self.shape[0]/self.features_per_lyr)
    
    def split(self,tensor):
        inputs=[tensor[:,(i%self.lyrs)*self.features_per_lyr:((i%self.lyrs)+1)*self.features_per_lyr] for i in range(len(self.hl_units))]
        return inputs

    def create_model(self,):
        input_=Input(shape=(self.time_steps,self.shape[0],),name='Input_layer')
        input_layer=Lambda(self.split)(input_)
        hidden_layers=[]
        hidden_layers.append(LSTM(self.hl_units[0],activation=LeakyReLU(alpha=0.3),return_sequences=True)(input_layer[0]))
        for i in range(1,len(self.hl_units)):
            merge=concatenate([input_layer[i],hidden_layers[i-1]])
            dropout=Dropout(0.1)(merge)
            hidden_layers.append(LSTM(self.hl_units[i],activation=LeakyReLU(alpha=0.3),return_sequences=True)(dropout))
        penultimate=concatenate(hidden_layers)
        ultimate=LSTM(32,activation=LeakyReLU(alpha=0.3))(penultimate)
        output=Dense(self.shape[1],activation='softmax')(ultimate)
        model45=Model(inputs=input_,outputs=output)
        return model45

In [9]:
def train_and_evaluate(train_index,test_index,fold):
    X_train, X_test = scaled_data[train_index], scaled_data[test_index]
    y_train, y_test = encoded_labels[train_index], encoded_labels[test_index]
    neurons=[50]*32
    model=SpinalLSTM((32,2),neurons,45,1)
    model=model.create_model()
    model.compile(loss=categorical_crossentropy,optimizer=Adam(learning_rate=0.001),metrics=[Accuracy()])
    model.fit(X_train,y_train,epochs=150)
    model.save(f'..model_saves/SpinalLSTM/PSD/model_valence/model_thread_{fold}')
    result=model.predict(X_test)
    new_result=[]
    for i in (result):
        new_result.append(np.where(i==i.max())[0])
    new_result=np.array(new_result).flatten()
    new_y=[]
    for i in (y_test):
        new_y.append(np.where(i==i.max())[0])
    new_y=np.array(new_y).flatten()
    accuracy = accuracy_score(new_y,new_result)
    return accuracy

In [10]:
k_fold = KFold(n_splits=32,shuffle=False)
with ThreadPoolExecutor() as executor:
    futures = []
    fold=0
    for train_index, test_index in k_fold.split(scaled_data):
        fold+=1
        futures.append(executor.submit(train_and_evaluate, train_index, test_index, fold))
    results = [future.result() for future in futures]

print("Cross-validation accuracies:", results)
print("Mean accuracy:", np.mean(results))






Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150
Epoch 1/150

39/39 [==============================] - 9s 223ms/step - loss: 0.6214 - accuracy: 0.0000e+00
Epoch 3/150
39/39 [==============================] - 8s 188ms/step - loss: 0.6298 - accuracy: 0.0000e+00
Epoch 3/150
39/39 [==============================] - 6s 145ms/step - loss: 0.6232 - accuracy: 0.0000e+00
Epoch 3/150
39/39 [==============================] - 7s 183ms/step - loss: 0.6025 - accuracy: 0.0000e+00
Epoch 4/150
39/39 [==============================] - 7s 178ms/step - loss: 0.5865 - accuracy: 0.0000e+00
Epoch 4/150
39/39 [==============================] - 6s 160ms/step - loss: 0.5649 - accuracy: 0.0000e+00
Epoch 5/150
39/39 [==============================] - 6s 156ms/step - loss: 0.5620 - accuracy: 0.0000e+00
Epoch 5/150
39/39 [==============================] - 4s 108ms/step - loss: 0.5567 - accuracy: 0.0000e+00
Epoch 5/150
39/39 [====

: 